# Исследование надежности заемщиков


In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца. 

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

Выведем перечень уникальных значений столбца `children`

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]


Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведим количество пропущенных значений для каждого столбца с помощью двух методов

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. 

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создаем в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [20]:
def categorize_purpose(row):
    try:
        if 'авто' in row:
            return 'операции с автомобилем'
        elif 'недвиж' in row or 'жиль' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'
    

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
print(data['purpose_category'].unique())

['операции с недвижимостью' 'операции с автомобилем'
 'получение образования' 'проведение свадьбы']


### Исследуем данные и ответимна вопросы

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
# проверим количество уникальных значений. В данных только бинарные значения. 1 - долг имеется, 0 - долга нет

data['debt'].value_counts() 

0    19599
1     1732
Name: debt, dtype: int64

In [23]:
# Количество должников менее 10%. Разберем какая категория с детьми входит в группу должников
children_debt = data.groupby('children')['debt'].value_counts()
print(children_debt)

children  debt
0         0       13028
          1        1063
1         0        4364
          1         444
2         0        1858
          1         194
3         0         303
          1          27
4         0          37
          1           4
5         0           9
Name: debt, dtype: int64


In [24]:
# в каждой группе должников определим соотношение должников к общему количеству в группе
children_debt = data.groupby('children').agg({'debt': ['count','sum']})
children_debt['ratio'] = round(children_debt['debt']['sum']/children_debt['debt']['count']*100,2)
children_debt

debt       ratio
          count   sum      
children                   
0         14091  1063  7.54
1          4808   444  9.23
2          2052   194  9.45
3           330    27  8.18
4            41     4  9.76
5             9     0  0.00

**Вывод:** В основном кредит берут люди без детей. В данной категории наблюдается наименьший процент задолженностей. В то время как в категории с детьми наблюдается обратная зависимость, чем больше детей, тем меньше берут кредит и именно в этих категориях наблюдается самая неблагоприятная ситуация с задолженностями, т.к. с увеличением кол-ва детей процент задолженности в категории увеличивается (за некоторым исключением, но данных по ним очень мало, поэтому целесообразно предположить, что уменьшение % задолженности или его отсутствие скорее исключение, а не статистика).
Из всего выше сказанного можно сделать вывод, что наличие детей и их кол-во влияет на своевременный возврат кредита (чем больше детей, тем больше вероятность увеличения процента должников).

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
# проверим количество уникальных значений. По большей части кредит берут люди, находящиеся в браке, около 57% всех заемщиков

data['family_status'].value_counts() 

женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

In [26]:
# сгруппируем данные по семейному статусу и определим количество должников в каждому факту долга - есть/нет 

family_debt = data.groupby('family_status')['debt'].value_counts()
print(family_debt)

family_status          debt
Не женат / не замужем  0        2523
                       1         273
в разводе              0        1105
                       1          84
вдовец / вдова         0         888
                       1          63
гражданский брак       0        3749
                       1         385
женат / замужем        0       11334
                       1         927
Name: debt, dtype: int64


In [27]:
# определим соотношение должников к количеству заемщиков в каждой группе
# и дополнительно соотношение данных к общеми кол-ву кредитов

family_debt = data.groupby('family_status').agg({'debt': ['count','sum']})
family_debt['ratio'] = round(family_debt['debt']['sum']/family_debt['debt']['count']*100,2)
family_debt['ratio_total'] = round(family_debt['debt']['sum']/family_debt['debt']['count'].sum()*100,2)
family_debt

debt      ratio ratio_total
                       count  sum                  
family_status                                      
Не женат / не замужем   2796  273  9.76        1.28
в разводе               1189   84  7.06        0.39
вдовец / вдова           951   63  6.62        0.30
гражданский брак        4134  385  9.31        1.80
женат / замужем        12261  927  7.56        4.35

**Вывод:** В основном кредит берут люди состоящие в браке (57%) и их процент задолженности находится практически на минимальном уровне среди изучаемых семейных статусов. Самые низкие показатели по задолженности у одиноких людей ранее состоящих в браке (в разводе или вдовцы), но они составляют всего около 10% всех заемщиков.
Самые неблагоприятные категории для своевременного возврата кредита, это люди не состоящие в браке или находящиеся в гражданском браке, возможно это молодая возрастная категория населения.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
# Получим статистику по доходам для должников и вовремя оплативших. 

data.groupby('debt')['total_income'].describe()


,count,mean,std,min,25%,50%,75%,max
debt,,,,,,,,
0,19599.0,165713.065105,98342.301226,21205.0,107440.50,142594.0,196614.5,2265604.0
1,1732.0,161152.278868,97923.730475,20667.0,108148.75,142594.0,187599.5,2200852.0


**Вывод:** Статистические данные показали, что средние, процентили, минимальные и максимальные значения отличаются незначительно. Из чего делаем вывод, что уровень дохода не является критерием для своевременной выплаты кредита

#### Как разные цели кредита влияют на его возврат в срок?

In [29]:
# проверим количество уникальных значений. 

data['purpose_category'].value_counts() 

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

In [30]:
# определим соотношение задолженности к общему количеству в каждой группе

purpose_debt = data.groupby('purpose_category').agg({'debt': ['count','sum']})
purpose_debt['ratio'] = round(purpose_debt['debt']['sum']/purpose_debt['debt']['count']*100,2)
purpose_debt['ratio_total'] = round(purpose_debt['debt']['sum']/purpose_debt['debt']['count'].sum()*100,2)
purpose_debt


debt      ratio ratio_total
                          count  sum                  
purpose_category                                      
операции с автомобилем     4279  400  9.35        1.88
операции с недвижимостью  10751  780  7.26        3.66
получение образования      3988  369  9.25        1.73
проведение свадьбы         2313  183  7.91        0.86

**Вывод:** В основном кредит берется на недвижимость (это 50% всех кредитов) и именно эта категория имеет наименьший процент должников. Далее по полулярности кредиты на авто и образование и именно в этих категориях наибольший процент задолженностей по кредиту. На последнем месте кредит на свадьбу (10% всех кредитов), но именно в данной категории низкий процент задолженности.

### Общий вывод.

Общий процент задолженности по погашению кредита составляет 8.12% за отчетный период. Исходя из полученных данным можно сделать вывод:
1. Количество детей влияет на факт погашения кредита вовремя. 
Основными  заемщиками банка являются люди без детей (66% всех кредитов). Наличие детей влияет на своевременность оплаты кредита: чем больше детей, тем больше процент должников в данной категории (процент должников возрастает с 7,54% до 9,76%)
2. Семейное положение влияет на факт погашения кредита вовремя. 
Основными  заемщиками банка являются люди, состоящие в браке (57% всех кредитов). И именно эта категория является одной из вовремяпогашающих кредит (процент задолженности в группе - 7,56%). Самые неблагоприятные категории для своевременного возврата кредита, это люди не состоящие в браке или находящиеся в гражданском браке, возможно это молодая возрастная категория населения (до 9,76% должников в группе)
3. Доход не влияет на факт погашения кредита вовремя.
Статистические данные показали, что уровень дохода не влияет на своевременность погашения кредита
4. Цель влияет на факт погашения кредита вовремя.
В основном кредит берется на недвижимость ( 50% всех кредитов) и именно эта категория самая вовремя погашающая кредит (7,26% должников в группе). В то время как на авто и образование наблюдается высокий уровень задолженности (до 9,35% в группе)